In [1]:
## plot score bedgraphs

suppressPackageStartupMessages({
library(data.table)
library(dplyr)
library(ggplot2)
library(stringr)
library(ggprism)
library(ggplot2)
library(cowplot)
library(dplyr)
library(Gviz)
library(rtracklayer)
library(BSgenome.Hsapiens.UCSC.hg38)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(GenomicRanges)
library("wesanderson")
    library(RColorBrewer)
    library(ggsci)
    library(ggrepel)
    })

### 

In [2]:
motifs <- fread('hocomoco_motifs.bed', header=FALSE)
colnames(motifs) <- c("chrom", "start", "end", "sequence","motif_class", "value")
motifs <- motifs %>% group_by(chrom, start,motif_class) %>% mutate(sequence = sequence[which.max(value)],
                                                            motif_class = motif_class[which.max(value)],
                                                            value = value[which.max(value)]) %>% ungroup() %>% unique()
## filter out low quality calls, only keep top 25% 
motifs <- filter(motifs, value > 0)
motifs <- filter(motifs, value > quantile(motifs$value, c(0.75)))

In [3]:
motifs_gata <- as.data.frame(motifs) %>% filter(grepl('GATA', motif_class)) %>% dplyr::select(chrom, start, end, value) %>% data.table() %>% unique()
motifs_gata <- motifs_gata %>% filter(value > 0)
motifs_bHLH <- as.data.frame(motifs) %>% filter(grepl('MYOD', motif_class) | 
                                               grepl('MYOG', motif_class) | 
                                               grepl('CLOCK', motif_class) | 
                                                grepl('ATOH', motif_class) | 
                                               grepl('BHA', motif_class) | 
                                                grepl('HIF', motif_class) | 
                                                grepl('ITF', motif_class) |
                                                grepl('MAX', motif_class) | 
                                               grepl('BHE', motif_class) |
                                                grepl('ASCL', motif_class) |
                                                grepl('LYL', motif_class) |
                                               grepl('MYC', motif_class)
                                               ) %>% dplyr::select(chrom, start, end, value) %>% data.table()%>% unique()
motifs_bHLH <- motifs_bHLH %>% filter(value > 0)

motifs_ETS <- as.data.frame(motifs) %>% filter(grepl('ETS', motif_class)) %>% dplyr::select(chrom, start, end, value) %>% data.table()%>% unique()
motifs_ETS <- motifs_ETS %>% filter(value > 0)

motifs_TCF <- as.data.frame(motifs) %>% filter(grepl('TCF', motif_class)) %>% dplyr::select(chrom, start, end, value) %>% data.table()%>% unique()
motifs_TCF <- motifs_TCF %>% filter(value > 0)

motifs_NFKB <- as.data.frame(motifs) %>% filter(grepl('NFKB', motif_class)) %>% dplyr::select(chrom, start, end, value) %>% data.table()%>% unique()
motifs_NFKB <- motifs_NFKB %>% filter(value > 0)

motifs_STAT <- as.data.frame(motifs) %>% filter(grepl('STAT', motif_class)) %>% dplyr::select(chrom, start, end, value) %>% data.table()%>% unique()
motifs_STAT <- motifs_STAT %>% filter(value > 0)

In [4]:
gtrack <- GenomeAxisTrack()
chr_no <- 12 
chr_start <- 9764556  # start of region
chr_end <- 9765505 # end of region

gen <- 'hg38'

gata_bedgraph <- DataTrack(range = motifs_gata,type = c("gradient"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                            end=chr_end, name = "Input*Gradient",col="red",showColorBar=TRUE,
                             ylim = c(0,15))
bHLH_bedgraph <- DataTrack(range = motifs_bHLH,type = c("gradient"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                            end=chr_end, name = "Input*Gradient",col="red",showColorBar=TRUE,
                             ylim = c(0,15))
TCF_bedgraph <- DataTrack(range = motifs_TCF,type = c("gradient"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                            end=chr_end, name = "Input*Gradient",col="red",showColorBar=TRUE,
                             ylim = c(0,15))
Ets_bedgraph <- DataTrack(range = motifs_ETS,type = c("gradient"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                            end=chr_end, name = "Input*Gradient",col="red",showColorBar=TRUE,
                             ylim = c(0,15))
NFKB_bedgraph <- DataTrack(range = motifs_NFKB,type = c("gradient"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                            end=chr_end, name = "Input*Gradient",col="red",showColorBar=TRUE,
                             ylim = c(0,15) )
STAT_bedgraph <- DataTrack(range = motifs_STAT,type = c("gradient"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                            end=chr_end, name = "Input*Gradient",col="red",showColorBar=TRUE,
                             ylim = c(0,15) )
pdf("motif_heatmap.pdf")
plotTracks(list(gtrack,
                gata_bedgraph,
                bHLH_bedgraph,
                TCF_bedgraph,
                Ets_bedgraph,
                NFKB_bedgraph,
                STAT_bedgraph),
           chromosome = "chr12",from = 9764556, to= 9765505,
           type = c("gradient"),
           ncolor=50,
           gradient=c("black","black")
           )
dev.off()

Warning message in .prepareDtData(data, len = length(range)):
"The following non-numeric data column has been dropped: chrom"
Warning message in .prepareDtData(data, len = length(range)):
"The following non-numeric data column has been dropped: chrom"
Warning message in .prepareDtData(data, len = length(range)):
"The following non-numeric data column has been dropped: chrom"
Warning message in .prepareDtData(data, len = length(range)):
"The following non-numeric data column has been dropped: chrom"
Warning message in .prepareDtData(data, len = length(range)):
"The following non-numeric data column has been dropped: chrom"
Warning message in .prepareDtData(data, len = length(range)):
"The following non-numeric data column has been dropped: chrom"


png 
  2

In [6]:
### read in be and dcas9 results
### fetch aggregated base editor data(averaged across replicates)

be <- fread('../rBE_data.tsv',header=FALSE)
colnames(be) <- c("start", "data")
be$chromosome <- "chr12"
be$end <- be$start + 20 ## guides are 20 bp long
be$data <- -1 * be$data ### flip 
be <- na.omit(be)

### fetch dcas9 krab data

### fetch aggregated base editor data(averaged across replicates)

dcas9 <- fread('~/BE_CD69_paper_2022/data/experimental/dCas9_data.tsv',header=TRUE)
colnames(dcas9) <- c("start", "data", "rep")
dcas9 <- dcas9 %>% group_by(start) %>% summarise(data = mean(data))
dcas9$chromosome <- "chr12"
dcas9$end <- dcas9$start + 20
dcas9$data <- -1 * dcas9$data
dcas9 <- na.omit(dcas9)




ERROR: Error in fread("~/BE_CD69_paper_2022//data/experimental/BE_data.tsv", : File '~/BE_CD69_paper_2022//data/experimental/BE_data.tsv' does not exist or is non-readable. getwd()=='/home/jupyter/repos/BE_CD69_paper_2022/Figure2'


In [7]:
## now lets plot the BE and dCas9 results
gtrack <- GenomeAxisTrack()
chr_no <- 12 
chr_start <- 9764556  # start of region
chr_end <- 9765505 # end of region

gen <- 'hg38'

be_bedgraph <- DataTrack(range = data.frame(be),type = c("p"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                         end=chr_end, name = "base editor",col="red")

dcas9_bedgraph <- DataTrack(range = data.frame(dcas9),type = c("p"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                         end=chr_end, name = "dcas9",col="red")

pdf("be_dcas9_results.pdf")
plotTracks(list(gtrack,
                be_bedgraph,
               dcas9_bedgraph), 
           chromosome = "chr12",from = 9764556, to= 9765505,
           ncolor=50,
           gradient=c("black","black")
           )
dev.off()


ERROR: Error in data.frame(be): object 'be' not found


In [ ]:
## overlay above w/ following heatmap to get feature width. plot remaining in illustrator

be_bedgraph <- DataTrack(range = data.frame(be),type = c("heatmap"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                         end=chr_end, name = "base editor",col="red")

dcas9_bedgraph <- DataTrack(range = data.frame(dcas9),type = c("heatmap"),genome = 'hg38',chromosome=chr_no,start=chr_start, 
                         end=chr_end, name = "dcas9",col="red")

#pdf("motif_heatmap.pdf")
plotTracks(list(gtrack,
                be_bedgraph,
               dcas9_bedgraph), 
           chromosome = "chr12",from = 9764556, to= 9765505,
           ncolor=50,
           gradient=c("black","black")
           )